In [38]:
from IPython.display import display, Image, clear_output

In [39]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets

import warnings

import time
##Custom scripts
# from OzESI_parsing_function_old import *

# from plotting_functions import *

# dfdfplotting_functions


In [40]:
# ### Filter for o3on and o2only
# # path to Projects/04_29_23/mzml folder

# import os
# import shutil
# print(os.getcwd())
# # Set the source directory where the files are located
# src_directory = "Projects/canola/" # For example, '/path/to/your/files'

# # Set the destination directories for o3on and o2only
# dst_directory_o3on = os.path.join(src_directory, "o3on")
# dst_directory_o2only = os.path.join(src_directory, "o2only")

# # Create the destination directories if they don't exist
# os.makedirs(dst_directory_o3on, exist_ok=True)
# os.makedirs(dst_directory_o2only, exist_ok=True)

# # Loop through the files in the source directory
# for filename in os.listdir(src_directory):
#     # Check if the file ends with ".mzML"
#     if filename.endswith(".mzML"):
#         # Determine which folder to move the file to based on its name
#         if "_o3on.mzML" in filename:
#             shutil.move(os.path.join(src_directory, filename), os.path.join(dst_directory_o3on, filename))
#         elif "_o2only.mzML" in filename:
#             shutil.move(os.path.join(src_directory, filename), os.path.join(dst_directory_o2only, filename))





In [41]:
data_base_name_location = 'lipid_database/Lipid_Database.xlsx'####Lipid database with Standard Carnitines
#NEW WAY
Pre_folder = './Projects/'
Project_Folder =Pre_folder + 'canola/mzml/'
folder_name_to_save = 'canola/results/'
file_name_to_save = 'canola'


tolerance = 0.3
remove_std = True
save_data= True


In [42]:

###All functions

#Function to read in MRM database
#Option to remove STDs from database##Not finished need option to use another database with no qualitative ACs


def read_mrm_list(filename,remove_std = True):
    mrm_list_new = pd.read_excel(filename, sheet_name=None)
    mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
    mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
    # Add underscore to middle of columns names
    mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
    # Round Parent Ion and Product Ion to 1 decimal place
    mrm_list_offical['Parent_Ion'] = np.round(mrm_list_offical['Parent_Ion'],1)
    mrm_list_offical['Product_Ion'] = np.round(mrm_list_offical['Product_Ion'],1)
    # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
    mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
    # Change column compound name to lipid
    mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
    # Make a column called Class match lipid column to lipid types
    if remove_std == True:
        lipid_class = mrm_list_offical['Class'].unique()
        lipid_class_to_keep = ['PS','PG','CE','PC', 'DAG', 'PE', 'TAG', 'FA', 'Cer', 'CAR', 'PI','SM']
        mrm_list_offical = mrm_list_offical[mrm_list_offical['Class'].isin(lipid_class_to_keep)]
    return mrm_list_offical


# Function to create an ion dictionary from an MRM database DataFrame
def create_ion_dict(mrm_database):
    ion_dict = defaultdict(list)
    # Iterate through the rows of the MRM database DataFrame
    for index, row in mrm_database.iterrows():
        # Add a tuple with Lipid and Class to the ion dictionary using Parent_Ion and Product_Ion as the key
        ion_dict[(row['Parent_Ion'], row['Product_Ion'])].append((row['Lipid'], row['Class']))
    return ion_dict

### New way to parse OzESI data
OzESI_time_df = pd.DataFrame(columns=['Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class', 'Sample_ID', 'Retention_Time', 'OzESI_Intensity'])


def mzml_parser(file_name):
    global OzESI_time_df  # Declare OzESI_time_df as a global variable
    
    rows = []
    ozesi_rows = []
    
    data_folder = os.listdir(file_name)
    data_folder.sort()
    path_to_mzml_files = file_name

    for file in data_folder:
        if file.endswith('.mzML'):
            run = pymzml.run.Reader(path_to_mzml_files + file, skip_chromatogram=False)
            q1_mz = 0
            q3_mz = 0

            for spectrum in run:
                for element in spectrum.ID.split(' '):
                    
                    if 'Q1' in element:
                        q1 = element.split('=')
                        q1_mz = np.round(float(q1[1]), 1)
                    
                    if 'Q3' in element:
                        q3 = element.split('=')
                        q3_mz = np.round(float(q3[1]), 1)
                        
                        intensity_store = np.array([intensity for _, intensity in spectrum.peaks()])
                        intensity_sum = np.sum(intensity_store)
                        
                        transition = f"{q1_mz} -> {q3_mz}"
                        sample_id = file[:-5]
                        
                        rows.append({
                            'Parent_Ion': q1_mz,
                            'Product_Ion': q3_mz,
                            'Intensity': intensity_sum,
                            'Transition': transition,
                            'Sample_ID': sample_id
                        })
                        
                        for time, intensity in spectrum.peaks():
                            ozesi_rows.append({
                                'Parent_Ion': q1_mz,
                                'Product_Ion': q3_mz,
                                'Retention_Time': time,
                                'OzESI_Intensity': intensity,
                                'Sample_ID': sample_id,
                                'Transition': transition
                            })

    df = pd.DataFrame(rows)
    OzESI_time_df = pd.DataFrame(ozesi_rows)
    print('Finished parsing mzML files\n')
    return df


# Function to check if the absolute difference between two values is within a given tolerance
def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance

# Function to match the ions in a DataFrame row with the ions in an ion dictionary
def match_ions(row, ion_dict, tolerance=0.3):
    ions = (row['Parent_Ion'], row['Product_Ion'])
    matched_lipids = []
    matched_classes = []

    # Iterate through the ion dictionary
    for key, value in ion_dict.items():
        # Check if both the Parent_Ion and Product_Ion values are within the specified tolerance
        if within_tolerance(ions[0], key[0], tolerance) and within_tolerance(ions[1], key[1], tolerance):
            # If within tolerance, extend the matched_lipids and matched_classes lists with the corresponding values
            matched_lipids.extend([match[0] for match in value])
            matched_classes.extend([match[1] for match in value])

    # If any matches were found, update the Lipid and Class columns in the row
    if matched_lipids and matched_classes:
        row['Lipid'] = ' | '.join(matched_lipids)
        row['Class'] = ' | '.join(matched_classes)

    return row

####Combined functions for Matching

def match_lipids_parser(mrm_database,df, tolerance=0.3):
    ion_dict = create_ion_dict(mrm_database)
    # Assuming you have the df DataFrame to apply the match_ions function
    df_matched = df.apply(lambda row: match_ions(row, ion_dict=ion_dict, tolerance=tolerance), axis=1)


    # df_matched = df_matched.dropna()
    
    return df_matched


def save_dataframe(df, folder_name, file_name, max_attempts=5):
    folder_path = f'data_results/data/data_matching/{folder_name}'
    os.makedirs(folder_path, exist_ok=True)

    for i in range(max_attempts):
        file_path = f'{folder_path}/{file_name}.csv'
        if not os.path.isfile(file_path):
            df.to_csv(file_path, index=False)
            print(f"Saved DataFrame to {file_path}")
            break
    else:
        print(f"Failed to save DataFrame after {max_attempts} attempts.")
        return None


def full_parse(data_base_name_location,mzml_folder, folder_name_to_save, file_name_to_save,tolerance,remove_std = True,
               save_data=False):
    mrm_database = read_mrm_list(data_base_name_location,remove_std=remove_std)
    df = mzml_parser(mzml_folder)
    df_matched = match_lipids_parser(mrm_database,df, tolerance=tolerance)
    
    if save_data == True:
        
        save_dataframe(df_matched, folder_name_to_save, file_name_to_save)

    return df_matched





In [43]:
df_matched = full_parse(data_base_name_location,Project_Folder, folder_name_to_save, 
                        file_name_to_save,tolerance, remove_std = remove_std,save_data=save_data)



/tmp/ipykernel_32369/388244400.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_32369/388244400.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_32369/388244400.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Finished parsing mzML files

Failed to save DataFrame after 5 attempts.


In [44]:

df_matched.head(None)
OzESI_time_df.head(None)



,Parent_Ion,Product_Ion,Retention_Time,OzESI_Intensity,Sample_ID,Transition
0,760.6,571.6,0.015933,1522.720093,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
1,760.6,571.6,0.032250,1262.600098,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
2,760.6,571.6,0.048550,1008.480042,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
3,760.6,571.6,0.064867,944.220093,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
4,760.6,571.6,0.081167,930.120056,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
...,...,...,...,...,...,...
600947,904.8,605.6,34.931717,791.740051,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6
600948,904.8,605.6,34.948017,766.820068,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6
600949,904.8,605.6,34.964317,737.020081,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6
600950,904.8,605.6,34.980633,719.400024,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6


In [45]:
mrm_database = read_mrm_list(data_base_name_location)
tolerance = 0.3
df_oz_matched = match_lipids_parser(mrm_database, OzESI_time_df, tolerance)


/tmp/ipykernel_32369/388244400.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_32369/388244400.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_32369/388244400.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
df_oz_matched.head(None)
# #print to excel
# df_oz_matched.to_excel("OzESI_time_df_04_29_23.xlsx")

,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
0,NaN,NaN,1522.720093,760.6,571.6,0.015933,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
1,NaN,NaN,1262.600098,760.6,571.6,0.032250,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
2,NaN,NaN,1008.480042,760.6,571.6,0.048550,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
3,NaN,NaN,944.220093,760.6,571.6,0.064867,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
4,NaN,NaN,930.120056,760.6,571.6,0.081167,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
...,...,...,...,...,...,...,...,...
600947,TAG,"[TG(55:9),TG(54:2)]_FA18:1",791.740051,904.8,605.6,34.931717,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6
600948,TAG,"[TG(55:9),TG(54:2)]_FA18:1",766.820068,904.8,605.6,34.948017,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6
600949,TAG,"[TG(55:9),TG(54:2)]_FA18:1",737.020081,904.8,605.6,34.964317,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6
600950,TAG,"[TG(55:9),TG(54:2)]_FA18:1",719.400024,904.8,605.6,34.980633,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,904.8 -> 605.6


In [47]:
#### Filter retention times

filtered_df = df_oz_matched[(df_oz_matched['Retention_Time'] > 10.0) & (df_oz_matched['Retention_Time'] < 16.0)]
filtered_df['Retention_Time'] = filtered_df['Retention_Time'].round(2)
filtered_df['OzESI_Intensity'] = filtered_df['OzESI_Intensity'].round(0)
# Assuming the DataFrame is named df

# Keep only unique combinations of 'Sample_ID' and 'Transition'
#filtered_df = filtered_df.drop_duplicates(subset=['Sample_ID', 'Transition'])


#filtered_df = filtered_df.groupby('Transition').apply(lambda x: x.loc[x['OzESI_Intensity'].idxmax()]).reset_index(drop=True)
filtered_df = filtered_df.groupby(['Sample_ID', 'Transition']).apply(lambda x: x.loc[x['OzESI_Intensity'].idxmax()]).reset_index(drop=True)
filtered_df.head(None)


/tmp/ipykernel_32369/2212824233.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_32369/2212824233.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
0,NaN,NaN,2244.0,760.6,571.6,15.64,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6
1,NaN,NaN,2608.0,762.6,573.6,15.46,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,762.6 -> 573.6
2,NaN,NaN,2100.0,764.6,575.6,15.70,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,764.6 -> 575.6
3,NaN,NaN,2049.0,766.7,577.6,15.93,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,766.7 -> 577.6
4,NaN,NaN,1510.0,782.6,593.6,14.20,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,782.6 -> 593.6
...,...,...,...,...,...,...,...,...
267,TAG,[TG(54:6)]_FA18:1,24887.0,896.8,597.6,10.77,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,896.8 -> 597.6
268,TAG,[TG(54:5)]_FA18:1,106327.0,898.8,599.6,12.59,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,898.8 -> 599.6
269,TAG,"[TG(55:11),TG(54:4)]_FA18:1",425121.0,900.8,601.6,14.34,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,900.8 -> 601.6
270,TAG,"[TG(55:10),TG(54:3)]_FA18:1",208996.0,902.8,603.6,14.32,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,902.8 -> 603.6


In [48]:
#Create df_matched_2 and concat df_matched and filtered_df
df_matched_2 = pd.concat([df_matched, filtered_df[['Retention_Time', 'OzESI_Intensity']]], axis=1)


In [49]:
df_matched_2.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity
0,NaN,3.655455e+06,NaN,760.6,571.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6,15.64,2244.0
1,NaN,4.520304e+06,NaN,762.6,573.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,762.6 -> 573.6,15.46,2608.0
2,NaN,3.607649e+06,NaN,764.6,575.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,764.6 -> 575.6,15.70,2100.0
3,NaN,3.323874e+06,NaN,766.7,577.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,766.7 -> 577.6,15.93,2049.0
4,NaN,2.403051e+06,NaN,782.6,593.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,782.6 -> 593.6,14.20,1510.0
...,...,...,...,...,...,...,...,...,...
275,TAG,2.772084e+06,[TG(54:6)]_FA18:1,896.8,597.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,896.8 -> 597.6,NaN,NaN
276,TAG,1.104765e+07,[TG(54:5)]_FA18:1,898.8,599.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,898.8 -> 599.6,NaN,NaN
277,TAG,1.144405e+07,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,900.8 -> 601.6,NaN,NaN
278,TAG,2.923619e+07,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,902.8 -> 603.6,NaN,NaN


In [50]:
# Create a new DataFrame to store the DB_Position and Aldehyde_Ion values
df_DB_position = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

# Loop over the range of DB_Position values and calculate the corresponding Aldehyde_Ion values
for i in range(3, 21):
    df_DB_position.loc[i,'DB_Position'] = i
    df_DB_position.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3)) 

# df_DB_position.head(None)

OzESI_list = [7,9,12]

for i in OzESI_list:
        # Retrieve the Aldehyde_Ion value for the current DB_Position
        aldehyde_ion = df_DB_position.loc[df_DB_position["DB_Position"] == i, "Aldehyde_Ion"].values[0]
        
        # Calculate n-i values
        df_matched_2["n-{}".format(i)] = df_matched_2["Parent_Ion"] - aldehyde_ion

df_matched_2.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12
0,NaN,3.655455e+06,NaN,760.6,571.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,760.6 -> 571.6,15.64,2244.0,678.6,650.6,608.6
1,NaN,4.520304e+06,NaN,762.6,573.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,762.6 -> 573.6,15.46,2608.0,680.6,652.6,610.6
2,NaN,3.607649e+06,NaN,764.6,575.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,764.6 -> 575.6,15.70,2100.0,682.6,654.6,612.6
3,NaN,3.323874e+06,NaN,766.7,577.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,766.7 -> 577.6,15.93,2049.0,684.7,656.7,614.7
4,NaN,2.403051e+06,NaN,782.6,593.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,782.6 -> 593.6,14.20,1510.0,700.6,672.6,630.6
...,...,...,...,...,...,...,...,...,...,...,...,...
275,TAG,2.772084e+06,[TG(54:6)]_FA18:1,896.8,597.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,896.8 -> 597.6,NaN,NaN,814.8,786.8,744.8
276,TAG,1.104765e+07,[TG(54:5)]_FA18:1,898.8,599.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,898.8 -> 599.6,NaN,NaN,816.8,788.8,746.8
277,TAG,1.144405e+07,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,900.8 -> 601.6,NaN,NaN,818.8,790.8,748.8
278,TAG,2.923619e+07,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,902.8 -> 603.6,NaN,NaN,820.8,792.8,750.8


In [51]:
#Make a new column for Labels of n-#
df_matched_2['Labels'] = ''
OzESI_list = [7, 9, 12]

def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance
columns = [
    'Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class',
    'Sample_ID', 'Retention_Time', 'Intensity_OzESI', 'Mean_Retention_Time',
    'Mean_Intensity_OzESI', 'n-7', 'n-9', 'n-12', 'Labels'
]
df_OzESI_n = pd.DataFrame(columns=columns)


#Function to add lipid name
def add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3):
    df_test = df_matched_2.copy()
    df_test_2 = df_matched_2.copy()
    global df_OzESI_n
    
    for i in OzESI_list:
        df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
    for i in range(len(df_test)):
        if pd.isna(df_test.loc[i, 'Lipid']):
            parent_ion = df_test.loc[i, 'Parent_Ion']
            
            for j in range(len(df_test)):
                row_data = df_test.loc[j].copy()
                if within_tolerance(parent_ion, row_data['n-7'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-7' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-7' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-9'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-9' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-9' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-12'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-12' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-12' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
    
    df_test_2.dropna(subset=['Lipid'], inplace=True)
    return df_test_2


In [52]:
#Add lipid name to df_matched 3
df_matched_3 = add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3)

/tmp/ipykernel_32369/642337914.py:46: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_32369/642337914.py:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [53]:
df_matched_3.head(None) 

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
23,TAG,2.284160e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.58,1567.0,786.7,758.7,716.7,
24,TAG,2.221219e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.38,2317.0,786.7,758.7,716.7,
25,TAG,3.482054e+06,[TG(52:5)]_FA18:1,870.8,571.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,870.8 -> 571.6,14.50,2398.0,788.8,760.8,718.8,
26,TAG,3.416310e+06,[TG(52:4)]_FA18:1,872.8,573.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,872.8 -> 573.6,15.72,14908.0,790.8,762.8,720.8,
27,TAG,2.042668e+07,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,874.8 -> 575.6,15.95,23449.0,792.8,764.8,722.8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2388,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2389,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2390,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7


In [54]:
df_matched_3_sorted = df_matched_3.sort_values(by=['Sample_ID','Product_Ion'])
df_matched_3_sorted.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
23,TAG,2.284160e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.58,1567.0,786.7,758.7,716.7,
24,TAG,2.221219e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.38,2317.0,786.7,758.7,716.7,
320,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7
321,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7
322,NaN,1.791068e+06,[TG(54:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2388,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2389,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2390,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7


In [55]:
df_matched_3_sorted.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
23,TAG,2.284160e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.58,1567.0,786.7,758.7,716.7,
24,TAG,2.221219e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.38,2317.0,786.7,758.7,716.7,
320,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7
321,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7
322,NaN,1.791068e+06,[TG(54:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2388,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2389,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7
2390,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7


In [56]:
df_matched_4 = df_matched_3_sorted.copy()
df_matched_4['Ratios'] = None


def calculate_intensity_ratio(df):
    # Create a new column for ratios
    df['Ratios'] = pd.Series(dtype='float64')

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        lipid = row['Lipid']
        label = row['Labels']
        intensity = row['OzESI_Intensity']
        sample_id = row['Sample_ID']

        # Check if the label is n-9
        if label == 'n-9':
            # Find the corresponding row with n-7 label and same lipid name
            n7_row = df[(df['Lipid'] == lipid) & (df['Labels'] == 'n-7')& (df['Sample_ID'] == sample_id)]

            # If a matching row is found, calculate the intensity ratio
            if not n7_row.empty:
                n7_intensity = n7_row['OzESI_Intensity'].values[0]
                ratio = intensity / n7_intensity

                # Assign the ratio to the 'Ratios' column
                df.at[index, 'Ratios'] = ratio

    return df

df_matched_4 = calculate_intensity_ratio(df_matched_4)


In [57]:
df_matched_4.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
23,TAG,2.284160e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.58,1567.0,786.7,758.7,716.7,,NaN
24,TAG,2.221219e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.38,2317.0,786.7,758.7,716.7,,NaN
320,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7,NaN
321,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7,NaN
322,NaN,1.791068e+06,[TG(54:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-9,0.805341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN
2388,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN
2389,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN
2390,NaN,2.937705e+06,"[TG(55:9),TG(54:2)]_FA18:1",822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN


In [58]:
#filtering df_matched for name but not removing the n-7 ratios
def sort_by_second_tg(lipid):
    tgs = lipid.split(',')
    if len(tgs) > 1:
        return tgs[1]
    else:
        return lipid

df_matched_4c = df_matched_4.copy()
df_matched_4c['Lipid'] = df_matched_4c['Lipid'].apply(sort_by_second_tg)
df_matched_4c.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
23,TAG,2.284160e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.58,1567.0,786.7,758.7,716.7,,NaN
24,TAG,2.221219e+06,[TG(52:6)]_FA18:1,868.7,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,868.7 -> 569.6,15.38,2317.0,786.7,758.7,716.7,,NaN
320,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7,NaN
321,NaN,1.791068e+06,[TG(52:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-7,NaN
322,NaN,1.791068e+06,[TG(54:6)]_FA18:1,786.6,569.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,786.6 -> 569.6,14.40,1146.0,704.6,676.6,634.6,n-9,0.805341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NaN,2.937705e+06,TG(54:2)]_FA18:1,822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN
2388,NaN,2.937705e+06,TG(54:2)]_FA18:1,822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN
2389,NaN,2.937705e+06,TG(54:2)]_FA18:1,822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN
2390,NaN,2.937705e+06,TG(54:2)]_FA18:1,822.7,605.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,822.7 -> 605.6,10.77,24887.0,740.7,712.7,670.7,n-7,NaN


In [59]:
# Iterate through each row in the DataFrame
for index, row in df_matched_4c.iterrows():
    # Extract Lipid, Sample_ID, Labels and Ratios from the row
    lipid = row['Lipid']
    sample_id = row['Sample_ID']
    labels = row['Labels']
    ratios = row['Ratios']

    # Check if ratios is not NaN
    if not pd.isna(ratios):
        # Print out the values
        print(f'Lipid: {lipid}, Sample_ID: {sample_id}, Labels: {labels}, Ratios: {ratios}')


Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.8053408292340126
Lipid: [TG(54:6)]_FA18:1, Sample_ID: TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723, Labels: n-9, Ratios: 0.805340829

In [60]:
def filter_highest_ratios(df):
    # Sort the DataFrame by ratios in descending order
    df_sorted = df.sort_values(by='Ratios', ascending=False)

    # Drop duplicates keeping the first occurrence (highest ratio)
    df_filtered = df_sorted.drop_duplicates(subset=['Sample_ID', 'Lipid','Labels'], keep='first')
    df_filtered = df_filtered.sort_values(by=['Sample_ID', 'Lipid'], ascending=[True, True])

    return df_filtered

df_matched_4d = filter_highest_ratios(df_matched_4c)


In [61]:
df_matched_4d.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
309,NaN,3.323874e+06,TG(52:2)]_FA18:1,766.7,577.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,766.7 -> 577.6,15.93,2049.0,684.7,656.7,614.7,n-9,1.287869
28,TAG,2.550369e+07,TG(52:2)]_FA18:1,876.8,577.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,876.8 -> 577.6,15.84,3469.0,794.8,766.8,724.8,,NaN
480,NaN,2.551021e+06,TG(52:2)]_FA18:1,794.7,577.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,794.7 -> 577.6,15.60,1591.0,712.7,684.7,642.7,n-7,NaN
301,NaN,3.607649e+06,TG(52:3)]_FA18:1,764.6,575.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,764.6 -> 575.6,15.70,2100.0,682.6,654.6,612.6,n-9,1.764706
27,TAG,2.042668e+07,TG(52:3)]_FA18:1,874.8,575.6,TailoredTAG18-1_O3off_Blank50ACN50IPA_v4_020723,874.8 -> 575.6,15.95,23449.0,792.8,764.8,722.8,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,TAG,1.104765e+07,[TG(54:5)]_FA18:1,898.8,599.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,898.8 -> 599.6,NaN,NaN,816.8,788.8,746.8,,NaN
2360,NaN,2.850856e+06,[TG(54:5)]_FA18:1,816.7,599.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,816.7 -> 599.6,14.37,50995.0,734.7,706.7,664.7,n-7,NaN
2194,NaN,2.655103e+06,[TG(54:6)]_FA18:1,786.6,597.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,786.6 -> 597.6,15.34,2627.0,704.6,676.6,634.6,n-9,0.157901
275,TAG,2.772084e+06,[TG(54:6)]_FA18:1,896.8,597.6,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,896.8 -> 597.6,NaN,NaN,814.8,786.8,744.8,,NaN


Visualizations

In [62]:
import plotly.express as px
import os

# Create the "Plots" directory if it doesn't exist
os.makedirs("Projects/canola/plots/", exist_ok=True)

#name of the folder to save plots
project_plots = "Projects/canola/plots/"

# Define color mappings for Lipid patterns
color_mapping = {
    '50': 'red',
    '51': 'brown',
    '52': 'blue',
    '53': 'purple',
    '54': 'green',
}

# Get the unique Sample_IDs
sample_ids = df_matched_4d['Sample_ID'].unique()

# Loop over the unique Sample_IDs
for sample_id in sample_ids:
    
    # Filter the dataframe for the current Sample_ID
    df_sample = df_matched_4d[df_matched_4d['Sample_ID'] == sample_id]
    
    # Assign colors to Lipids based on patterns
    lipid_colors = []
    for lipid in df_sample['Lipid']:
        color = 'gray'  # Default color
        for pattern, pattern_color in color_mapping.items():
            if pattern in lipid:
                color = pattern_color
                break
        lipid_colors.append(color)
    
    # Create the bar plot
    fig = px.bar(df_sample, x='Lipid', y='Ratios', text='Ratios', title=f'Bar Plot for Sample_ID: {sample_id}',
                 color_discrete_map=color_mapping)
    
    # Apply colors to the bars
    fig.update_traces(
        marker_color=lipid_colors,
        texttemplate='%{text:.2f}',
        textposition='auto',
        marker_line_width=0
    )
    
    # Customize the layout
    fig.update_layout(
        uniformtext_minsize=18,
        uniformtext_mode='hide',
        xaxis=dict(
            title='Lipid',
            titlefont=dict(size=16)
        ),
        yaxis=dict(
            title='Ratios',
            titlefont=dict(size=16),
            tickfont=dict(size=16)  # Set the font size of y-axis labels
        ),
        legend=dict(
            title='Lipid Patterns',
            tracegroupgap=50,
            itemsizing='constant'
        ),
        title=dict(
            text=f'Sample_ID: {sample_id}',
            font=dict(size=20)  # Set the title font size
        )
    )
    # Save the plot as an image
    file_name = project_plots + f"/plot_{sample_id}.png"

    # Check if the file already exists
    index = 1
    while os.path.exists(file_name):
        file_name = project_plots + f"/plot_{sample_id}_{index}.png"
        index += 1

    fig.write_image(file_name)
    # Show plot
    fig.show()


In [63]:
#########################

In [64]:
##########################
##########################
##########################
##########################
##########################
##########################

Caitlin Data

In [65]:
import pandas as pd


val_df = pd.read_excel("Caitlin.xlsx", header=1)
val_df = val_df.rename(columns={'Ratio n-9/n-7': 'Ratios'})
val_df.head(None)

,Lipid,MRM Identity,MRM Transition,RT,Relative Abundance %,n-9 isomer %,n-7 isomer %,Ratios,Sample_ID
0,TG 50:1,TG 50:1_FA 18:1,850.8→551.6,15,16.441215,67.910997,32.089003,2.116332,DOD93 5xFAD F4 Hippocampus
1,TG 50:2,TG 50:2_FA 18:1,850.8→549.6,13.8,4.155703,76.970166,23.029834,3.342194,DOD93 5xFAD F4 Hippocampus
2,TG 50:3,TG 50:3_FA 18:1,846.8→547.6,12.5,0.375821,NaN,NaN,NaN,DOD93 5xFAD F4 Hippocampus
3,TG 52:2,TG 52:2_FA 18:1,876.8→547.6,14.8,41.588725,69.849531,30.150469,2.316698,DOD93 5xFAD F4 Hippocampus
4,TG 52:3,TG 52:3_FA 18:1,874.8→575.6,13.7,7.943199,77.032611,22.967389,3.354000,DOD93 5xFAD F4 Hippocampus
...,...,...,...,...,...,...,...,...,...
112,TG 52:3,TG 52:3_FA 18:1,874.8→575.6,13.7,12.187151,87.409386,12.590614,6.942424,FAD 184 F4 WT Cerebellum
113,TG 52:4,TG 52:4_FA 18:1,872.8→573.6,nd,1.459308,NaN,NaN,NaN,FAD 184 F4 WT Cerebellum
114,TG 54:3,TG 54:3_FA 18:1,902.8→603.6,14.7,22.415748,88.152261,11.847739,7.440429,FAD 184 F4 WT Cerebellum
115,TG 54:4,TG 54:4_FA 18:1,900.8→601.6,13.5,10.833710,100.000000,0.000000,NaN,FAD 184 F4 WT Cerebellum


In [66]:
import plotly.express as px

# Assuming val_df is your DataFrame after renaming the column
fig = px.scatter(val_df, x="Lipid", y="Ratios", color="Sample_ID",
                 title="Lipid vs Ratios for each Sample_ID",
                 labels={"Lipid": "Lipid", "Ratios": "Ratios"},
                 hover_data=["Sample_ID"])

fig.show()



In [67]:
import pandas as pd

# Adding a new column 'Person' to indicate the source of data
val_df['Person'] = 'Caitlin'
df_matched_4d['Person'] = 'Me'

# Selecting only the required columns
val_df = val_df[['Lipid', 'Ratios', 'Sample_ID', 'Person']]
df_matched_4d = df_matched_4d[['Lipid', 'Ratios', 'Sample_ID', 'Person']]

# Concatenating the two DataFrames
validation = pd.concat([val_df, df_matched_4d], ignore_index=True)

# Displaying the combined DataFrame
validation.head(None)

,Lipid,Ratios,Sample_ID,Person
0,TG 50:1,2.116332,DOD93 5xFAD F4 Hippocampus,Caitlin
1,TG 50:2,3.342194,DOD93 5xFAD F4 Hippocampus,Caitlin
2,TG 50:3,NaN,DOD93 5xFAD F4 Hippocampus,Caitlin
3,TG 52:2,2.316698,DOD93 5xFAD F4 Hippocampus,Caitlin
4,TG 52:3,3.354000,DOD93 5xFAD F4 Hippocampus,Caitlin
...,...,...,...,...
368,[TG(54:5)]_FA18:1,NaN,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me
369,[TG(54:5)]_FA18:1,NaN,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me
370,[TG(54:6)]_FA18:1,0.157901,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me
371,[TG(54:6)]_FA18:1,NaN,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me


In [68]:

import re
# Mapping for 3-letter codes
keywords_mapping = {
    'hippocampus': 'HIP',
    'hippo': 'HIP',
    'cortex': 'COR',
    'cort': 'COR',
    'cerebellum': 'CER',
    'cereb': 'CER',
    'diencephalon': 'DIE',
    'dienc': 'DIE'
}
additional_keywords = ['DOD93', 'DOD94', 'FAD185', 'FAD189']

def get_id(sample_id):
    # Iterate over the keywords_mapping dictionary
    for keyword, id_code in keywords_mapping.items():
        # Check if the keyword is present in the sample_id
        if keyword.lower() in sample_id.lower():
            return id_code
    
    # Return None if no match is found
    return None

# Function to get additional ID based on keywords
def get_additional_id(sample_id):
    for keyword in additional_keywords:
        if keyword.lower() in sample_id.lower():
            return keyword
    return None

# Create the new 'ID' column
validation['ID'] = validation['Sample_ID'].apply(get_id)

# Create the new 'ID_2' column
validation['ID_2'] = validation['Sample_ID'].apply(get_additional_id)




def reformat_lipid_name(lipid_name):
    # Find pattern like TG(50:1) within the string
    match = re.search(r'\b([A-Z]{2})\((\d+:\d+)\)', lipid_name)
    # If the pattern is found, reformat it as 'TG 50:1'
    if match:
        return f"{match.group(1)} {match.group(2)}"
    # If the pattern is not found, return the original string
    return lipid_name

# Apply the transformation to each entry in the 'Lipid' column
validation['Lipid'] = validation['Lipid'].apply(reformat_lipid_name)
def extract_f3_f4(sample_id):
    # Check if 'F3', 'F4', 'M1', 'M2', 'M3', 'M4', or 'Ms' is in the Sample_ID and return it
    if 'F3' in sample_id:
        return 'F3'
    elif 'F4' in sample_id:
        return 'F4'
    elif 'M1' in sample_id:
        return 'M1'
    elif 'M2' in sample_id:
        return 'M2'
    elif 'M3' in sample_id:
        return 'M3'
    elif 'M4' in sample_id:
        return 'M4'
    elif 'Ms' in sample_id:
        return 'Ms'
    else:
        return None


# Create a new column 'ID_3' in the DataFrame
validation['ID_3'] = validation['Sample_ID'].apply(extract_f3_f4)


# validation.to_excel("validation.xlsx")
# Display the DataFrame
validation.head(None)

,Lipid,Ratios,Sample_ID,Person,ID,ID_2,ID_3
0,TG 50:1,2.116332,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
1,TG 50:2,3.342194,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
2,TG 50:3,NaN,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
3,TG 52:2,2.316698,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
4,TG 52:3,3.354000,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
...,...,...,...,...,...,...,...
368,TG 54:5,NaN,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me,None,None,None
369,TG 54:5,NaN,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me,None,None,None
370,TG 54:6,0.157901,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me,None,None,None
371,TG 54:6,NaN,TailoredTAG18-1_O3on_RBDCanola0.0005mgmL_020723,Me,None,None,None


In [69]:
import plotly.express as px
save_dir = "Projects/04_29_23/Plots/validation/"
# Fill NaN values in the 'ID', 'ID_2', and 'ID_3' columns with a default value
validation['ID'] = validation['ID'].fillna('UNKNOWN')
validation['ID_2'] = validation['ID_2'].fillna('UNKNOWN')
validation['ID_3'] = validation['ID_3'].fillna('UNKNOWN')

# Loop through unique combinations of ID, ID_2, and ID_3
unique_ids = validation[['ID', 'ID_2', 'ID_3']].drop_duplicates().values

for id1, id2, id3 in unique_ids:
    # Filter the DataFrame for the current ID, ID_2, and ID_3
    filtered_df = validation[(validation['ID'] == id1) & (validation['ID_2'] == id2) & (validation['ID_3'] == id3)]

    # Create scatter plot for the current ID, ID_2, and ID_3
    fig = px.scatter(filtered_df, x="Lipid", y="Ratios", color="Person",
                     title=f"Lipid vs Ratios for ID: {id1}, ID_2: {id2}, ID_3: {id3}",
                     labels={"Lipid": "Lipid", "Ratios": "Ratios"},
                     hover_data=["Sample_ID"])
    # Save the figure as PNG
    save_path = os.path.join(save_dir, f"plot_{id1}_{id2}_{id3}.png")
    fig.write_image(save_path, format='png')

    # Display the figure
    fig.show()
